In [65]:
import pandas as pd
import os

def preprocess_df(df):
    df['A'] = df['A'].astype(str)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    if df.iloc[0]['A'] != 'Bundesgebiet insgesamt':
        df = df.iloc[1:]
    if df.iloc[-1]['A'] != 'Ausland':
        df = df.iloc[:-1]
    return df

directory = 'C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\1.9\\'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df = preprocess_df(df)
        dataframes.append(df)

if len(set(len(df) for df in dataframes)) > 1:
    raise ValueError("Not all dataframes have the same length")

all_data = pd.concat(dataframes, ignore_index=True)
rename_map = {'Bundesgebiet insgesamt': 'Gesamt', 'Schleswig Holstein': 'Schleswig-Holstein'}
all_data['A'] = all_data['A'].replace(rename_map)
all_data.rename(columns={'A': 'Region', 'B': 'ankuenfte_anzahl', 'C': 'ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'D': 'uebernachtungen_anzahl', 'E': 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'F': 'durchsch_aufenthaltsdauer_tage'}, inplace=True)

short_forms = ['Gesamt', 'Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
current_prefix = None

for index, row in all_data.iterrows():
    if row['Region'] in short_forms:
        current_prefix = row['Region']
    elif current_prefix:
        all_data.at[index, 'Region'] = current_prefix + '_' + row['Region']

all_data['land'] = all_data['Region'].apply(lambda x: x.split('_')[0] if '_' in x else x)
all_data['wohnsitz'] = all_data['Region'].apply(lambda x: x.split('_')[-1] if '_' in x else x)
all_data.drop(columns=['Region'], inplace=True)

all_data = all_data[['land', 'wohnsitz'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz']]]
all_data['wohnsitz'] = all_data['wohnsitz'].replace({'Deutschland': 'Inland'})
all_data['wohnsitz'] = all_data['wohnsitz'].apply(lambda x: x if x in ['Inland', 'Ausland'] else 'Gesamt')

all_data[['jahr', 'monat']] = all_data['Sequence'].astype(str).str.extract(r'(\d{4})(\d{2})')
all_data.drop(columns=['Sequence'], inplace=True)

month_map = {
    '01': 'Januar', '02': 'Februar', '03': 'März', '04': 'April', '05': 'Mai', '06': 'Juni',
    '07': 'Juli', '08': 'August', '09': 'September', '10': 'Oktober', '11': 'November', '12': 'Dezember'
}
all_data['monat'] = all_data['monat'].replace(month_map)

all_data = all_data[['land', 'wohnsitz', 'jahr', 'monat'] + [col for col in all_data.columns if col not in ['land', 'wohnsitz', 'jahr', 'monat']]]

int_columns = ['jahr', 'ankuenfte_anzahl', 'uebernachtungen_anzahl']
float_columns = ['ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent', 'uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent', 'durchsch_aufenthaltsdauer_tage']
all_data[int_columns] = all_data[int_columns].apply(pd.to_numeric, errors='coerce').fillna('.')
all_data[float_columns] = all_data[float_columns].apply(pd.to_numeric, errors='coerce').fillna('.')

all_data.to_csv('C:\\Users\\Ronja\\Documents\\Studium\\Semester 4\\DABI2\\Projekt\\Umsatzdaten\\DESTATIS\\Output\\combined_0901_2212_sheet_19.csv', index=False)


In [66]:
all_data.tail()

,land,wohnsitz,jahr,monat,ankuenfte_anzahl,ankuenfte_veraenderung_zum_vorjahreszeitraum_prozent,uebernachtungen_anzahl,uebernachtungen_veraenderung_zum_vorjahreszeitraum_prozent,durchsch_aufenthaltsdauer_tage
8512,Schleswig-Holstein,Inland,2022,Dezember,15055.0,-6.6,37614.0,-13.5,2.5
8513,Schleswig-Holstein,Ausland,2022,Dezember,264.0,69.2,558.0,76.0,2.1
8514,Thüringen,Gesamt,2022,Dezember,2792.0,283.0,6194.0,240.1,2.2
8515,Thüringen,Inland,2022,Dezember,2751.0,.,6054.0,.,2.2
8516,Thüringen,Ausland,2022,Dezember,41.0,.,140.0,.,3.4
